In [0]:
# Databricks notebook source
# MAGIC %run ./1_config.py

# COMMAND ----------

# MAGIC %run ./2_setup.py

# COMMAND ----------


spark.sql(f"USE CATALOG {conf.catalog}")
spark.sql(f"USE {conf.db_name}")

tables = [
    "fact_traffic_15min",
    "fact_daily_summary",
    conf.silver_table,
    "raw_traffic",
    "dim_time",
    "dim_detector",
    "region_lookup",
]

for t in tables:
    fqn = f"{conf.catalog}.{conf.db_name}.{t}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {fqn}")
        print("Dropped", fqn)
    except Exception as e:
        print("Skip drop", fqn, e)

# Nuke checkpoints/schemas for Auto Loader
base = conf.checkpoint_base.rstrip("/")
for sub in ["bronze", "silver", "gold"]:
    path = f"{base}/{sub}"
    try:
        dbutils.fs.rm(path, recurse=True)
        print("Removed", path)
    except Exception as e:
        print("Skip", path, e)

# (Optional) also remove Autoloader per-stream subfolders if you changed patterns
# dbutils.fs.rm(f"{base}/bronze/streaming", True)

# Recreate empty schema & tables
SH = SetupHelper(conf)
SH.setup()
SH.validate()
